## Chatbot Finance 
#### 01/2018

** To do**
* Date Range - Greg - plot 
* Weekends - last available - Milosz
* Returns - Michal +plot 
* When someone give a time range, produce a simple plot for - stock price on these days also scaled to daily returns. - Greg + MH
* Correlations - Michal Heydel
* Market return -what are weights for the given firms in S&P 500?

### Input:
* Date, Stock_symbol.
* Metrics: Stock Price, Revenue/Profit in fundamental data, Volatility - Sd/variance

* Later on: Correlation

### Output:
* What is asked. 

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter, YearLocator, MonthLocator
import matplotlib.dates as mdates
%matplotlib inline

import seaborn as sns
sns.reset_orig() # don't change the style of plots

import glob
import datetime
import warnings
warnings.filterwarnings('ignore')

In [3]:
csv_files = glob.glob('nyse\*.csv')
csv_files

['nyse\\fundamentals.csv',
 'nyse\\prices-split-adjusted.csv',
 'nyse\\prices.csv',
 'nyse\\securities.csv']

In [4]:
def read_stock_price_data():
    csv_files = glob.glob('nyse\*.csv')
    csv_files
    stock_price_df = pd.read_csv(csv_files[1], parse_dates = ['date'])
    return stock_price_df

In [5]:
#Main()
stock_price_df = read_stock_price_data()

In [6]:
import dateutil.parser
def stock_price_on_day(stock_symbol, datestring, data = stock_price_df):
    date = dateutil.parser.parse(datestring, dayfirst = True)
    ## code to say that date need to be between 2010-01-01 to 2016-12-30 -> raiseError if?, assert(??)
    return data.loc[(stock_price_df['symbol'] == stock_symbol) & (stock_price_df['date'] == date)]

In [7]:
# First problem...
print(dateutil.parser.parse("21/05/2016", dayfirst = True))
print(dateutil.parser.parse("04/05/2016", dayfirst = True))
print(dateutil.parser.parse("2013-05-01", dayfirst = True))
print(dateutil.parser.parse("05/01/2013"))
## looks like "dayfirst = True" solves the problem..

# Second problem - some days there is no data - for example the weekends.. 
## How to treat this..

2016-05-21 00:00:00
2016-05-04 00:00:00
2013-01-05 00:00:00
2013-05-01 00:00:00


In [8]:
stock_price_on_day('NFLX', "01/05/2013")

,date,symbol,open,close,low,high,volume
394311,2013-05-01,NFLX,30.845715,30.415714,30.235714,31.055714,18359600.0


In [9]:
stock_price_on_day('NFLX', "05/01/2013")
# Problem that is a weekend

,date,symbol,open,close,low,high,volume


In [10]:
def stock_price_on_day2(stock_symbol, datestring, data = stock_price_df):
    date = dateutil.parser.parse(datestring, dayfirst = True)
    ## code to say that date need to be between 2010-01-01 to 2016-12-30 -> raiseError if?, assert(??)    
    on_day_df = data.loc[(stock_price_df['symbol'] == stock_symbol) & (stock_price_df['date'] == date)]
    if on_day_df.empty:
        datestring = date - datetime.timedelta(days = 1)
        datestring = datestring.strftime('%d/%m/%Y')
        return stock_price_on_day2(stock_symbol, datestring, data = stock_price_df)
    else:
        comm = "showing latest available data for " + datestring
        return on_day_df, comm
### this function returns the latest available data when asked for a single day
### it gives a tuple with the dataframe with the data and a commend specifying the date 
### to draw the user's attention

In [11]:
stock_price_on_day2('NFLX', "06/01/2013")

(             date symbol       open      close        low       high  \
 355610 2013-01-04   NFLX  13.791429  13.711429  13.648571  13.958571   
 
             volume  
 355610  17761100.0  , 'showing latest available data for 04/01/2013')

In [12]:
def stock_price_on_range(stock_symbol, date_start, date_end, data = stock_price_df):    
    date_start = dateutil.parser.parse(date_start, dayfirst = True)
    date_end = dateutil.parser.parse(date_end, dayfirst = True)
    dates = pd.date_range(start=date_start, end=date_end, freq='D')
    ## code to say that date need to be between 2010-01-01 to 2016-12-30 -> raiseError if?, assert(??)
    prices_on_range_df = stock_price_on_day(stock_symbol, dates[0].strftime('%d/%m/%Y'))
    for i in range(1, len(dates)):
        next_date_df = stock_price_on_day('NFLX', dates[i].strftime('%d/%m/%Y'))
        prices_on_range_df = prices_on_range_df.append(next_date_df)
    return prices_on_range_df

### this function returns a dataframe of prices for a specified range of dates

In [13]:
stock_price_on_range('NFLX', "08/05/2013", "10/05/2013")

,date,symbol,open,close,low,high,volume
396731,2013-05-08,NFLX,29.275715,29.801428,29.218571,30.057142,17623200.0
397215,2013-05-09,NFLX,29.718571,30.915714,29.502857,31.268572,36108800.0
397699,2013-05-10,NFLX,30.915714,31.098572,30.651428,31.392857,18844700.0


In [16]:
stock_price_df.loc[(stock_price_df['symbol'] == 'NFLX') & (stock_price_df['date'] == '2011-01-06')]

,date,symbol,open,close,low,high,volume
120106,2011-01-06,NFLX,25.168571,25.427143,25.092857,25.67,31991400.0


In [21]:
stock_price_df.tail()

,date,symbol,open,close,low,high,volume
851259,2016-12-30,ZBH,103.309998,103.199997,102.849998,103.930000,973800.0
851260,2016-12-30,ZION,43.070000,43.040001,42.689999,43.310001,1938100.0
851261,2016-12-30,ZTS,53.639999,53.529999,53.270000,53.740002,1701200.0
851262,2016-12-30,AIV,44.730000,45.450001,44.410000,45.590000,1380900.0
851263,2016-12-30,FTV,54.200001,53.630001,53.389999,54.480000,705100.0


---
## 1. Stock Returns
---
### Calculating the Returns:

**Simple Total return ($R_{simple}$) over the 7-year period; and the return over the 7-year period ($r_{7}$)** (2010-01-01 to 2016-12-31):  
### $R_{simple} = 1 + r_{7}= (1+r_{annual})^7 =\frac{P_{t+7y}}{P_t}  => $  $r_{7} = (\frac{P_{t+7y}}{P_t} - 1) $

**Annualised Return:**  
### $r_{annual} = (\sqrt[7]{\frac{P_{t+7y}}{P_t}} - 1)   = \sqrt[7]{1 + r_{7}} -1$

**Daily logarithmic returns:**  
### $r_{daily} = log(P_t) - log(P_{t-1d})$, as $P_t = P_{t-1}*e^{r_{daily}}$
---

In [30]:
prices_df = stock_price_df[['symbol','close', 'date']].sort_values(['symbol', 'date'])
prices_long_df = prices_df.copy() # i will use this dataframe later
prices_df.head(3)

,symbol,close,date
251,A,22.389128,2010-01-04
718,A,22.145923,2010-01-05
1186,A,22.067240,2010-01-06


In [31]:
#pivoting
prices_df = prices_df.pivot_table(index = 'date', columns = 'symbol', values = 'close')

#Some manipulations
prices_df.reset_index(inplace=True)
prices_dates = prices_df['date']
prices_df.drop('date', inplace = True, axis = 1)
prices_df.head(3)

symbol,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,...,XLNX,XOM,XRAY,XRX,XYL,YHOO,YUM,ZBH,ZION,ZTS
0,22.389128,4.77,40.380001,30.572857,NaN,26.629999,26.129884,42.070000,37.090000,31.670000,...,25.379999,69.150002,35.330002,8.63,NaN,17.10,25.226457,60.020000,13.33,NaN
1,22.145923,5.31,40.139999,30.625713,NaN,26.440001,25.918773,42.330002,37.700001,31.620001,...,25.059999,69.419998,34.910000,8.64,NaN,17.23,25.140187,61.919998,13.80,NaN
2,22.067240,5.09,40.490002,30.138571,NaN,26.190001,26.062713,42.779999,37.619999,31.559999,...,24.889999,70.019997,35.139999,8.56,NaN,17.17,24.960462,61.900002,15.00,NaN


In [33]:
##DAILY LOGARITHMIC RETURNS
return_daily_df = prices_df.apply(lambda x: np.log(x) - np.log(x.shift(1))) # shift moves dates back by 1.
return_daily_df.set_index(prices_dates, inplace = True)
return_daily_df.head(5)

symbol,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,...,XLNX,XOM,XRAY,XRX,XYL,YHOO,YUM,ZBH,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-05,-0.010922,0.107246,-0.005961,0.001727,NaN,-0.007160,-0.008112,0.006161,0.016313,-0.001580,...,-0.012689,0.003897,-0.011959,0.001158,NaN,0.007574,-0.003426,0.031165,0.034651,NaN
2010-01-06,-0.003559,-0.042314,0.008682,-0.016034,NaN,-0.009500,0.005538,0.010575,-0.002124,-0.001899,...,-0.006807,0.008606,0.006567,-0.009302,NaN,-0.003488,-0.007175,-0.000323,0.083382,NaN
2010-01-07,-0.001298,0.029044,-0.000247,-0.001850,NaN,-0.016167,0.008250,-0.000935,-0.019595,-0.007953,...,-0.010095,-0.003147,0.013006,0.004662,NaN,-0.027755,-0.000288,0.022681,0.106160,NaN
2010-01-08,-0.000325,-0.019268,0.003945,0.006626,NaN,0.010807,0.005099,-0.003986,-0.005436,0.005733,...,0.014505,-0.004020,0.000000,-0.003494,NaN,0.000000,0.000288,-0.021228,-0.016319,NaN


In [36]:
### TOTAL Returns starting on the day the stock was first noted on the S&P500 index.
def func_1(x):
    '''Takes a column(stock name) and returns the value of the first notice/listing in the S&P500 and the index of the notice'''
    return prices_df.loc[prices_df[x].first_valid_index(),x], prices_df[x].first_valid_index()

# Create empty DataFrame
return_stocks_df = pd.DataFrame()

# go thorugh all the cols and calculate
for col in prices_df.columns:
    return_stocks_df[col] = prices_df[col] / func_1(col)[0] - 1
    
return_stocks_df.set_index(prices_dates, inplace = True)
return_stocks_df.head()

,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,...,XLNX,XOM,XRAY,XRX,XYL,YHOO,YUM,ZBH,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN
2010-01-05,-0.010863,0.113208,-0.005944,0.001729,NaN,-0.007135,-0.008079,0.006180,0.016447,-0.001579,...,-0.012608,0.003904,-0.011888,0.001159,NaN,0.007602,-0.003420,0.031656,0.035259,NaN
2010-01-06,-0.014377,0.067086,0.002724,-0.014205,NaN,-0.016523,-0.002571,0.016877,0.014290,-0.003473,...,-0.019307,0.012581,-0.005378,-0.008111,NaN,0.004094,-0.010544,0.031323,0.125281,NaN
2010-01-07,-0.015655,0.098532,0.002476,-0.016027,NaN,-0.032294,0.005692,0.015926,-0.005392,-0.011367,...,-0.029157,0.009400,0.007642,-0.003476,NaN,-0.023392,-0.010829,0.054982,0.251313,NaN
2010-01-08,-0.015974,0.077568,0.006439,-0.009486,NaN,-0.021780,0.010834,0.011885,-0.010785,-0.005684,...,-0.014972,0.005351,0.007642,-0.006952,NaN,-0.023392,-0.010544,0.032822,0.231058,NaN


In [39]:
return_7years_df = return_stocks_df["2016-12-30":].transpose().sort_values("2016-12-30", ascending = False) 
return_7years_df.columns = ['Return']
return_7years_df.head()

,Return
NFLX,15.204189
REGN,13.904182
ULTA,12.524669
URI,9.515936
ALK,9.178377


In [40]:
# Calculating the annualized returns
return_stocks_annual_df = ((1 + return_7years_df)**(1/7)) - 1 

return_stocks_annual_df.head()

,Return
NFLX,0.488689
REGN,0.471009
ULTA,0.450740
URI,0.399517
ALK,0.393010
